<a href="https://colab.research.google.com/github/radosty/radosty.github.io/blob/main/stockdatamarger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# Load the S&P 500 and NASDAQ dataset from the provided URL
url_sp500_nasdaq = 'https://raw.githubusercontent.com/radosty/radosty.github.io/refs/heads/main/s%26p500andnasdaqstock.csv'
sp500_nasdaq_data = pd.read_csv(url_sp500_nasdaq)

# Load the Fintech dataset from the provided URL
url_fintech = 'https://raw.githubusercontent.com/radosty/radosty.github.io/refs/heads/main/fintechstock2.csv'
fintech_data = pd.read_csv(url_fintech)

# Convert 'Date' columns to datetime format and remove timezone information
sp500_nasdaq_data['Date'] = pd.to_datetime(sp500_nasdaq_data['Date'], utc=True).dt.tz_convert(None)
fintech_data['Date'] = pd.to_datetime(fintech_data['Date'], utc=True).dt.tz_convert(None)

# Rename columns in fintech_data to match sp500_nasdaq_data
fintech_data.rename(columns={
    'Average Closing Price': 'Close',
    'Average Monthly Return (%)': 'Monthly Return (%)'
}, inplace=True)

# Add 'Ticker' column to fintech_data
fintech_data['Ticker'] = 'Fintech'

# Combine the datasets
merged_data = pd.concat([sp500_nasdaq_data, fintech_data], ignore_index=True)

# Add new rows for January 1, 2021
new_data = pd.DataFrame({
    'Date': [pd.to_datetime('2021-01-01'), pd.to_datetime('2021-01-01')],
    'Ticker': ['S&P 500', 'NASDAQ'],
    'Close': [3793.75, None],
    'Monthly Return (%)': [None, 0.87]
})

# Concatenate the new data with the existing dataset
merged_data = pd.concat([merged_data, new_data], ignore_index=True)

# Group by year, month, and ticker, then take the last available value
merged_data['Year'] = merged_data['Date'].dt.year
merged_data['Month'] = merged_data['Date'].dt.month
grouped_data = merged_data.groupby(['Year', 'Month', 'Ticker']).last().reset_index()

# Recreate the Date column
grouped_data['Date'] = pd.to_datetime(grouped_data['Year'].astype(str) + '-' + grouped_data['Month'].astype(str) + '-01')

# Drop the Year and Month columns
grouped_data = grouped_data.drop(['Year', 'Month'], axis=1)

# Sort the dataset by Date and Ticker
grouped_data = grouped_data.sort_values(['Date', 'Ticker'])

# Impute missing values in 'Monthly Return (%)' with the mean
mean_monthly_return = grouped_data['Monthly Return (%)'].mean()
grouped_data['Monthly Return (%)'].fillna(mean_monthly_return, inplace=True)
mean_closing = grouped_data['Close'].mean()
grouped_data['Close'].fillna(mean_closing, inplace= True)
# Save the updated dataset
grouped_data.to_csv('monthly_stock_data3.csv', index=False)

print("Monthly data has been saved to monthly_stock_data.csv")

# Display the first few rows of the final dataset
print(grouped_data.head(10))



Monthly data has been saved to monthly_stock_data.csv
    Ticker       Date         Close  Monthly Return (%)
0  Fintech 2020-12-01   1719.500000            1.806003
1  Fintech 2021-01-01   1916.000000           11.427741
2   NASDAQ 2021-01-01   6468.393088            0.870000
3  S&P 500 2021-01-01   3793.750000            1.806003
4  Fintech 2021-02-01   1903.500000           -0.652401
5   NASDAQ 2021-02-01  13192.349610            0.930779
6  S&P 500 2021-02-01   3811.149902            2.609145
7  Fintech 2021-03-01   2041.500000            7.249803
8   NASDAQ 2021-03-01  13246.870120            0.413274
9  S&P 500 2021-03-01   3972.889893            4.243863


<ipython-input-6-727c46864e17>:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  grouped_data['Monthly Return (%)'].fillna(mean_monthly_return, inplace=True)
<ipython-input-6-727c46864e17>:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val